Reference:

John R. Ladd, Jessica Otis, Christopher N. Warren, and Scott Weingart, "Exploring and Analyzing Network Data with Python," The Programming Historian 6 (2017), https://doi.org/10.46430/phen0064.

https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python

In [1]:
from collections import Counter
import functools
import itertools
from operator import itemgetter
import re
import timeit

import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy import stats

from utils import *

%matplotlib notebook

## Graph utilities

In [2]:
def add_node_attributes(G):
    """Compute various graph metrics and add to node attributes.
    """
    # degree
    nx.set_node_attributes(G,
                           dict(G.out_degree(G.nodes())), 
                           'out_degree')
    nx.set_node_attributes(G,
                           dict(G.in_degree(G.nodes())), 
                           'in_degree')
    
    if (type(G) != nx.classes.multidigraph.MultiDiGraph):
        # eigenvector centrality
        nx.set_node_attributes(G, 
                               nx.eigenvector_centrality(G),
                               'eigenvector')

        # betweenness centrality
        nx.set_node_attributes(G,
                               nx.betweenness_centrality(G), 
                               'betweenness')

        # degree centrality
        nx.set_node_attributes(G,
                               nx.degree_centrality(re),
                               'degree_centrality')

In [3]:
def top_nodes(data_dict, limit=20, show=False):
    """Return and optionally print the top n nodes in an attribute
    dictionary.
    """
    nodes = sorted(data_dict.items(), 
                   key=lambda item: item[1], 
                   reverse=True)[:limit]
    if show:
        print("Top {} nodes:".format(limit))
        for d in nodes:
            print(d)
        
    return nodes
        
def highest_value(attribute_dict):     
    """Find the node with largest value in an attribute dictionary.
    
    Return: 
        tuple(node, value)
        
    """     
    # Ordered tuple
    attr_items = [(b,a) for (a,b) in attribute_dict.iteritems()]    
    # Sort in descending order     
    attr_items.sort()     
    attr_items.reverse()     
    return tuple(reversed(attr_items[0]))

### Export

In [4]:
def to_df(G):
    """Export graph to Pandas dataframe with attributes as columns."""
    return pd.DataFrame(dict(G.nodes(data=True))).T
    
def to_file(G, file):
    """Export graph to .gexf file."""
    nx.write_gexf(G, file)
    
def to_txt(G, file):
    """Export node attributes to .txt file."""
    results = [(k, bet_cen[k], clo_cen[k], eig_cen[k]) for k in range(len(nodes))]
    
    f = open(file,'w')
    for item in results:     
        f.write(','.join(map(str,item)))     
        f.write('\n')
    f.close()

def get_matrix(G):
    return nx.to_numpy_matrix(G)

In [5]:
def visualize(G, format_dict=None):
    if format_dict == None:
        format_dict = {'font_size':16,
                       'width':3,
                       'edge_color':'grey',
                       'node_color':'purple',
                       'with_labels':False,
                      }

    fig, ax = plt.subplots(figsize=(10, 8))

    pos = nx.spring_layout(G, k=2)

    nx.draw_networkx(G, 
                     pos,
                     ax=ax,
                     kwargs=format_dict,
                    )
    # Offset labels
    for key, value in pos.items():
        x, y = value[0]+.135, value[1]+.045
        ax.text(x, y,
                s=key,
                bbox=dict(facecolor='red', alpha=0.25),
                horizontalalignment='center', fontsize=13)

In [6]:
def graph_properties(graph):
    properties = []
    out = "Nodes: {}".format(graph.number_of_nodes())
    properties.append(out)
    print(out)
    
    out = "Edges: {}".format(graph.number_of_edges())
    print(out)
    properties.append(out)
    
    degrees = [val for (node, val) in graph.degree()]
    out = "Maximum degree: {}".format(np.max(degrees))
    properties.append(out)
    print(out)
    
    out = "Minimum degree: {}".format(np.min(degrees))
    properties.append(out)
    print(out)
    
    out = "Average degree: {:.1f}".format(np.mean(degrees))
    properties.append(out)
    print(out)
    
    out = "Mode: {}".format(stats.mode(degrees)[0][0])
    properties.append(out)
    print(out)
    
    return properties

## Build User Graphs

### structure:
MultiDiGraph:
- nodes are userids $\rightarrow$ make nodes from unique ids
- edges are a tweet
- edges are directed (```user1 -retweet> user2```)
- multiple edges between nodes (```user1``` retweeted ```user2``` multiple times)

In [7]:
def get_unique_ids(campaign, campaign_users):
    campaign_userids = (campaign_users
           .groupby('userid')
           .size()
           .index
          )
    tweet_userids = (campaign
                   .groupby('userid')
                   .size()
                   .index
                  )
    retweet_userids = (campaign
                   .loc[:]['retweet_userid']
                   .dropna()
                  )
    reply_userids = (campaign
                   .loc[:]['in_reply_to_userid']
                   .dropna()
                  )
    mention_userids = (campaign['user_mentions']
                # expand usernames into columns
                .explode()
                # melt wide table into duplicated tweets
                .reset_index()
                .melt(id_vars=['tweetid'],
                      value_name='mentioned_userid')
                # clean up
                .astype({'mentioned_userid':'string'})
                .drop(columns=['variable'])
                .join(campaign['userid'], on='tweetid')
                .dropna()
               ).loc[:]['mentioned_userid']
    
    return Counter(list(itertools.chain(*[campaign_userids,
                                            tweet_userids,
                                            retweet_userids,
                                            reply_userids,
                                            mention_userids,
                                           ])))

In [8]:
def get_connections(tweets):
    """Generate three multi-digraphs of connections ((user1, user2), tweetid):
        1. Retweeted: ((['userid'], ['retweet_userid'])['tweetid'])
        2. Replied: ((['userid'], ['in_reply_to_userid']), ['tweetid'])
        3. Mentioned: ((['userid'], ['mentioned_userid']), ['tweetid'])
    """
    
    mention_userids = (campaign['user_mentions']
                # expand usernames into columns
                .explode()
                # melt wide table into duplicated tweets
                .reset_index()
                .melt(id_vars=['tweetid'],
                      value_name='mentioned_userid')
                # clean up
                .astype({'mentioned_userid':'string'})
                .drop(columns=['variable'])
                .join(campaign['userid'], on='tweetid')
                .dropna()
               )
    
    rt = (campaign
          .loc[:][['userid','retweet_userid']]
          .dropna()
          .reset_index()
         )
    re = (campaign
          .loc[:][['userid', 'in_reply_to_userid']]
          .dropna()
          .reset_index()
         )
    mentions = (mention_userids
                .loc[:][['userid','mentioned_userid','tweetid']]
                .query('mentioned_userid != ""')
               )
    
    unique_ids = dict(get_unique_ids(campaign, campaign_users))
    
    # retweets graph
    rt_graph = nx.MultiDiGraph()

    rt_graph.add_nodes_from(unique_ids.keys(), count=unique_ids.values())

    dictionary = [dict(tweetid=tid) for tid in rt['tweetid']]
    rt_graph.add_edges_from(list(zip(rt['userid'],
                                     rt['retweet_userid'],
                                     dictionary)))
    
    # replies graph
    re_graph = nx.MultiDiGraph()

    re_graph.add_nodes_from(unique_ids.keys(), count=unique_ids.values())

    dictionary = [dict(tweetid=tid) for tid in re['tweetid']]
    re_graph.add_edges_from(list(zip(re['userid'],
                                     re['in_reply_to_userid'],
                                     dictionary)))
    
    # mentions graph
    mention_graph = nx.MultiDiGraph()

    mention_graph.add_nodes_from(unique_ids.keys(), count=unique_ids.values())

    dictionary = [dict(tweetid=tid) for tid in mentions['tweetid']]
    mention_graph.add_edges_from(list(zip(mentions['userid'],
                                          mentions['mentioned_userid'],
                                          dictionary)))
    
    return rt_graph, re_graph, mention_graph

## Example: Iran 12/2020 Campaign

In [9]:
users = UsersData('data/users')
tweets = TweetsData('data/tweets')

In [10]:
tweets_df = tweets.df.loc[:][:]
users_df = users.df.loc[:][:]

In [11]:
campaign = tweets_df[tweets_df['campaign'] == 'iran202012']
campaign_users = users_df[users_df['campaign'] == 'iran202012']

In [12]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560571 entries, 1271764746983952390 to 948848104362663936
Data columns (total 36 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   userid                    560571 non-null  string        
 1   user_display_name         560571 non-null  string        
 2   user_screen_name          560571 non-null  string        
 3   user_reported_location    417523 non-null  string        
 4   user_profile_description  530518 non-null  string        
 5   user_profile_url          338535 non-null  string        
 6   follower_count            560571 non-null  int64         
 7   following_count           560571 non-null  int64         
 8   account_creation_date     560571 non-null  datetime64[ns]
 9   account_language          560571 non-null  string        
 10  tweet_language            444758 non-null  string        
 11  tweet_text                560571 no

In [13]:
rt_graph, re_graph, mention_graph = get_connections(campaign)

In [14]:
add_node_attributes(rt_graph)
add_node_attributes(re_graph)
add_node_attributes(mention_graph)

In [15]:
properties = graph_properties(rt_graph)

Nodes: 61117
Edges: 17981
Maximum degree: 9737
Minimum degree: 0
Average degree: 0.6
Mode: 0


In [16]:
most_retweets = top_nodes(nx.get_node_attributes(rt_graph, 'out_degree'),limit=20)
most_retweets

[('213589457', 4857),
 ('rrkP3RNDiwWqLySjcoqdFlz7DrFX7RCwwDSxyDqaU=', 2025),
 ('79Tf6XH3DwjdUWGO4aQWghSj5G2esetBnePoOBB3wYM=', 1522),
 ('z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=', 1453),
 ('BYaaZkKxjVQjhsnn9REZ0UcFoEHl+tKnzJ+0Hv+Pg=', 1336),
 ('6qhrzJLryTiE7VlJkmY+cKE5VsITiaFwMA7s3Dr5I=', 649),
 ('794097607845015552', 625),
 ('QXyUcPxvoQ+uzkqCj6O+AT9nUf3Avh0JTvwTG9GncQk=', 581),
 ('A2ufeZxtC32uuqgSKfm1EiJPs4uMP3aM7AfHUMsTM=', 571),
 ('Hz3n1y2y3sLOknEGxPWczYyCSeZrz15JLPTcJAd6oAo=', 569),
 ('EOkiPW6kT+GE5A2wU5mqPe7hiK8s7Cr0tfCybFsz2Sg=', 523),
 ('ihPCMQ32xEzpD35Et9IH4HO21XKiWdSJreVg+pHT5o=', 403),
 ('1091795789263921152', 371),
 ('etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=', 293),
 ('LubegkQHPmV20BdQonItq03iWY5unAkvlI2M7PNKvjI=', 241),
 ('1067814896706994176', 211),
 ('xkCWfX+FKOlOHNYjW1jeAG4IfrkxhOZLVbgnDWHVevg=', 158),
 ('Bl9IUxp6GStTkiNGbhfBJM9xu85e3Y9BQdNd97GFAI=', 155),
 ('6UQXw4Mnf0c4YkJ5rSJWtgfZlNBQFdx38mOzYUt16sI=', 133),
 ('T8AyWQK3ajadJJL2J+OJovSr4z+rpoK4LqM6ob+wTw=', 124)

In [17]:
most_retweeted = top_nodes(nx.get_node_attributes(rt_graph, 'in_degree'),limit=20)
most_retweeted

[('213589457', 4880),
 ('Bl9IUxp6GStTkiNGbhfBJM9xu85e3Y9BQdNd97GFAI=', 2623),
 ('1067814896706994176', 2407),
 ('etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=', 1783),
 ('794097607845015552', 625),
 ('BYaaZkKxjVQjhsnn9REZ0UcFoEHl+tKnzJ+0Hv+Pg=', 545),
 ('79Tf6XH3DwjdUWGO4aQWghSj5G2esetBnePoOBB3wYM=', 544),
 ('z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=', 529),
 ('6qhrzJLryTiE7VlJkmY+cKE5VsITiaFwMA7s3Dr5I=', 505),
 ('rrkP3RNDiwWqLySjcoqdFlz7DrFX7RCwwDSxyDqaU=', 445),
 ('1091795789263921152', 371),
 ('ihPCMQ32xEzpD35Et9IH4HO21XKiWdSJreVg+pHT5o=', 366),
 ('mpY+AZI80Nu61VS3o4Zm+UXTjaxrkE08xU0nO3JQ=', 342),
 ('EOkiPW6kT+GE5A2wU5mqPe7hiK8s7Cr0tfCybFsz2Sg=', 299),
 ('1099221870530961408', 215),
 ('XCVp3AesS42sNbyxhBsKM62AylqUJyipi3laS53gY=', 168),
 ('6UQXw4Mnf0c4YkJ5rSJWtgfZlNBQFdx38mOzYUt16sI=', 133),
 ('QXyUcPxvoQ+uzkqCj6O+AT9nUf3Avh0JTvwTG9GncQk=', 133),
 ('xkCWfX+FKOlOHNYjW1jeAG4IfrkxhOZLVbgnDWHVevg=', 109),
 ('T8AyWQK3ajadJJL2J+OJovSr4z+rpoK4LqM6ob+wTw=', 106)]

In [18]:
fields = ['user_display_name',
           'user_reported_location',
           'user_profile_description',
           'follower_count',
           'following_count']

common_users = [userid for userid in dict(most_retweeted).keys() 
                if userid in dict(most_retweets).keys()]
(campaign_users
 .loc[common_users][fields]
)

,user_display_name,user_reported_location,user_profile_description,follower_count,following_count
userid,,,,,
213589457,HispanTV,Teherán,"Medio no corporativo, que se hace eco de las r...",161512,31
Bl9IUxp6GStTkiNGbhfBJM9xu85e3Y9BQdNd97GFAI=,Bl9IUxp6GStTkiNGbhfBJM9xu85e3Y9BQdNd97GFAI=,<NA>,Freelance Journalist 📝 Middle-Eastern Politics...,2499,2701
1067814896706994176,Fatma Nursani,"Jakarta Barat, DKI Jakarta",أَمَّنْ يُجِيبُ الْمُضْطَرَّ إِذَا دَعَاهُ وَي...,5292,2384
etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=,etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=,Jakarta,"Pejuang keadilan 💪✊🇮🇩 Yang belum kefollow, in...",4436,2613
794097607845015552,Political Economy,<NA>,<NA>,5578,596
BYaaZkKxjVQjhsnn9REZ0UcFoEHl+tKnzJ+0Hv+Pg=,BYaaZkKxjVQjhsnn9REZ0UcFoEHl+tKnzJ+0Hv+Pg=,"Jakarta Selatan, DKI Jakarta",Account that fought for the oppressed countrie...,2485,1619
79Tf6XH3DwjdUWGO4aQWghSj5G2esetBnePoOBB3wYM=,79Tf6XH3DwjdUWGO4aQWghSj5G2esetBnePoOBB3wYM=,"Jakarta Selatan, DKI Jakarta",Indonesian 🇮🇩 😘 Middle Eastern Activist 💪🏻 Ant...,2536,2419
z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=,z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=,<NA>,in God we Trust,745,794
6qhrzJLryTiE7VlJkmY+cKE5VsITiaFwMA7s3Dr5I=,6qhrzJLryTiE7VlJkmY+cKE5VsITiaFwMA7s3Dr5I=,Indonesia,sharing my happy thoughts 🌹,584,383


In [19]:
most_replies = top_nodes(nx.get_node_attributes(re_graph, 'out_degree'),limit=20)
most_replies

[('0zCl5U0pYu0gEmK3JtjO5fbnxEj6pO9GUgH52Q6yg0E=', 13282),
 ('T8AyWQK3ajadJJL2J+OJovSr4z+rpoK4LqM6ob+wTw=', 10258),
 ('etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=', 8118),
 ('XpT328jUSYUJJM4Bt0HmKSJ8d8rJWgkBkQhdff62skE=', 6296),
 ('WFMM2eW8YAMex3B1z4P8vxx0OvkEJvFCx9l0uYGMqug=', 5969),
 ('1091795789263921152', 5086),
 ('1067814896706994176', 4852),
 ('wqcue0aByZjWGy2aKzh3IiKOK3urEeCGmRPpbpfvPBc=', 2703),
 ('1099221870530961408', 1832),
 ('rrkP3RNDiwWqLySjcoqdFlz7DrFX7RCwwDSxyDqaU=', 1774),
 ('z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=', 1746),
 ('mpY+AZI80Nu61VS3o4Zm+UXTjaxrkE08xU0nO3JQ=', 1427),
 ('DsqKOBIbyhdb1fRCREnRRabr2ijPMzNgximA+u8N8=', 1233),
 ('iVnWPNBGsoPoOI0d759LSmf2u0DGfCCHSznVuHLNjo=', 1218),
 ('lTQFNj2OGHS3yOz2SZimhktwdgrQmMsXUuaGHi7zek=', 1125),
 ('tfIRka6bXJoCm84UisMb2m4L8XpKyPJr7Yfv8UmeQ=', 1109),
 ('6UQXw4Mnf0c4YkJ5rSJWtgfZlNBQFdx38mOzYUt16sI=', 1075),
 ('5O68KDqdztRXhsl9HkNpWLfdwjINlgxlCDiKbBCFzzc=', 982),
 ('wBrHiyqOL5uBhytBGwy2W3A8nDXyRNfgAXfqXn3e0GY=', 982),
 ('eb

In [20]:
most_replied = top_nodes(nx.get_node_attributes(re_graph, 'in_degree'),limit=20)
most_replied

[('25073877', 6440),
 ('759251', 5264),
 ('rrkP3RNDiwWqLySjcoqdFlz7DrFX7RCwwDSxyDqaU=', 1686),
 ('z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=', 1666),
 ('47448563', 693),
 ('EOkiPW6kT+GE5A2wU5mqPe7hiK8s7Cr0tfCybFsz2Sg=', 635),
 ('2467791', 608),
 ('2836421', 540),
 ('QXyUcPxvoQ+uzkqCj6O+AT9nUf3Avh0JTvwTG9GncQk=', 529),
 ('138212728', 513),
 ('790728', 487),
 ('410718052', 475),
 ('128262354', 420),
 ('irXcL8Y80STibhjlS3a48BahFUg3yqPT+XtTxkpLL8I=', 350),
 ('BYaaZkKxjVQjhsnn9REZ0UcFoEHl+tKnzJ+0Hv+Pg=', 344),
 ('1285276850411864065', 316),
 ('5bXQzilfbR70wndojgGtIlSmUsL8W8RbAPssGLY0Y=', 302),
 ('1009577803304656896', 298),
 ('818910970567344128', 295),
 ('374729426', 291)]

In [21]:
common_users = [userid for userid in dict(most_replied).keys() 
                if userid in dict(most_replies).keys()]
(campaign_users
 .loc[common_users][fields]
)

,user_display_name,user_reported_location,user_profile_description,follower_count,following_count
userid,,,,,
rrkP3RNDiwWqLySjcoqdFlz7DrFX7RCwwDSxyDqaU=,rrkP3RNDiwWqLySjcoqdFlz7DrFX7RCwwDSxyDqaU=,<NA>,Treat others the way you'd like to be treated ...,2911,2441
z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=,z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=,<NA>,in God we Trust,745,794


In [29]:
most_mentioning = top_nodes(nx.get_node_attributes(mention_graph, 'out_degree'),limit=20)
most_mentioning

[('213589457', 42752),
 ('1235289416370774016', 23375),
 ('0zCl5U0pYu0gEmK3JtjO5fbnxEj6pO9GUgH52Q6yg0E=', 23120),
 ('T8AyWQK3ajadJJL2J+OJovSr4z+rpoK4LqM6ob+wTw=', 22923),
 ('etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=', 14665),
 ('1067814896706994176', 11811),
 ('WFMM2eW8YAMex3B1z4P8vxx0OvkEJvFCx9l0uYGMqug=', 8936),
 ('XpT328jUSYUJJM4Bt0HmKSJ8d8rJWgkBkQhdff62skE=', 8023),
 ('A8dbNc6R+dK0Ml02nrs1gjJKz9HIHRqLkcIixDiREoM=', 7997),
 ('1091795789263921152', 6760),
 ('fOKnPXaJon5pH9ZoHgWWQG+L1TdWue7y5OAPSPhLZs=', 5706),
 ('rrkP3RNDiwWqLySjcoqdFlz7DrFX7RCwwDSxyDqaU=', 5413),
 ('79Tf6XH3DwjdUWGO4aQWghSj5G2esetBnePoOBB3wYM=', 5110),
 ('z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=', 4412),
 ('6qhrzJLryTiE7VlJkmY+cKE5VsITiaFwMA7s3Dr5I=', 4296),
 ('ihPCMQ32xEzpD35Et9IH4HO21XKiWdSJreVg+pHT5o=', 3923),
 ('BYaaZkKxjVQjhsnn9REZ0UcFoEHl+tKnzJ+0Hv+Pg=', 3868),
 ('1099221870530961408', 3479),
 ('wqcue0aByZjWGy2aKzh3IiKOK3urEeCGmRPpbpfvPBc=', 3433),
 ('794097607845015552', 3380)]

In [30]:
most_mentioned = top_nodes(nx.get_node_attributes(mention_graph, 'in_degree'),limit=20)
most_mentioned

[('10228272', 34405),
 ('25073877', 7660),
 ('759251', 5409),
 ('irXcL8Y80STibhjlS3a48BahFUg3yqPT+XtTxkpLL8I=', 4901),
 (' 25073877', 3219),
 ('Bl9IUxp6GStTkiNGbhfBJM9xu85e3Y9BQdNd97GFAI=', 2744),
 ('5bXQzilfbR70wndojgGtIlSmUsL8W8RbAPssGLY0Y=', 2573),
 ('etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=', 1964),
 ('1172112779815399424', 1759),
 ('1252764865', 1688),
 ('150573137', 976),
 ('1640929196', 727),
 ('47448563', 700),
 (' 939091', 694),
 ('1212304197627998209', 675),
 ('2467791', 647),
 ('1215636956291510274', 626),
 ('iMu6Ukwlry6PiiRnPSE710AKRlyOVlncYVaHAY9tRU4=', 625),
 ('1175350204628787200', 611),
 ('1009577803304656896', 601)]

In [31]:
common_users = [userid for userid in dict(most_mentioned).keys() 
                if userid in dict(most_mentioning).keys()]
(campaign_users
 .loc[common_users][fields]
)

,user_display_name,user_reported_location,user_profile_description,follower_count,following_count
userid,,,,,
etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=,etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=,Jakarta,"Pejuang keadilan 💪✊🇮🇩 Yang belum kefollow, in...",4436,2613


### Density, clustering, transitivity

Measuring the tendency for nodes to cluster together or for edges to form triangles: this corresponds to measures of the extent to which the users interacting with one particular user tend to interact with each other as well. These metrics can provide insight into how users tend to create groups characterized by dense connections.

- Density describes the ratio of graph edges to the number of edges if the graph were fully connected.

- The clustering coefficient is calculated as the number of triangles connected to node $i$ divided by the number of sets of two edges connected to node $i$ (node triples). 

- The transitivity coefficient is calculated as 3 times the number of triangles in the network, divided by the number of connected triples of nodes in the network. Transitivity weights nodes with a large degree higher.

In [22]:
digraph = nx.DiGraph(rt_graph)

In [23]:
print('Retweets:')
out = 'Density: {}'.format(nx.density(digraph))
print(out)
out = "Average clustering coefficient: {}".format(nx.average_clustering(digraph))
print(out)
out = "Transitivity: {}".format(nx.transitivity(digraph))
print(out)

Retweets:
Density: 8.540311915515041e-08
Average clustering coefficient: 0.00037529907717083943
Transitivity: 0.6254199328107503


### Other attributes

In [24]:
def add_undirected_attributes(G):
    
    G.graph['connected_components'] = nx.number_connected_components(graph)
    
    clustering_coeffs = nx.clustering(graph)
    G.graph['avg_clust'] = (sum(clustering_coeffs.values()) 
                             / len(clustering_coeffs)) 

In [25]:
graph = nx.Graph(rt_graph)
add_undirected_attributes(graph)

In [26]:
out = 'Connected components: {}'.format(nx.number_connected_components(graph))
print(out)

clustering_coeffs = nx.clustering(graph)
out = 'Avg. clustering coefficient: {}'.format(sum(clustering_coeffs.values()) 
                             / len(clustering_coeffs))
print(out)

Connected components: 61056
Avg. clustering coefficient: 0.0004933720185999085


### Centrality

Capture the importance of a node's position in the network considering: 
- degree, on the assumption that an important node will have many connections
- closeness, on the assumption that important nodes are close to other nodes

In [27]:
graph_centrality = nx.degree_centrality(graph)
sorted(graph_centrality.items(), 
       key=lambda item: item[1],
       reverse=True)[:5]

[('v6groR3jb3Pkm5X9ccSwgoPnmlZzKEkx5bsc1XQHb0=', 0.0003108842201714772),
 ('etDaWEjMPleueDrpkatPUSCApc6yU8W95+yZYWzVxSY=', 0.00027815956541658485),
 ('1067814896706994176', 0.00026179723803913867),
 ('79Tf6XH3DwjdUWGO4aQWghSj5G2esetBnePoOBB3wYM=', 0.00026179723803913867),
 ('Bl9IUxp6GStTkiNGbhfBJM9xu85e3Y9BQdNd97GFAI=', 0.00026179723803913867)]

In [28]:
graph_closeness = nx.closeness_centrality(graph)
sorted(graph_closeness.items(),
       key=lambda item: item[1],
       reverse=True)[:5]

[('y3KkURpZFjT+WeW9e6BcxBYRg311F8fz1eJ647ahQc=', 0.00033293176299362923),
 ('wBrHiyqOL5uBhytBGwy2W3A8nDXyRNfgAXfqXn3e0GY=', 0.0003220159674856414),
 ('79Tf6XH3DwjdUWGO4aQWghSj5G2esetBnePoOBB3wYM=', 0.00029989273307823094),
 ('z3nCVBEHiIbcBhhxU2mOz5iWK4a7sUdGmRSPFM16G0=', 0.00029762081843369886),
 ('v6groR3jb3Pkm5X9ccSwgoPnmlZzKEkx5bsc1XQHb0=', 0.0002931787166660317)]

## To do
- In general, these measures reflect the sparsity of connections among users in the total user graph. Further analysis could search for denser subsets of the overall graph.
- Removal of outliers, e.g. for more meaningful statistics on in/out degree